# LinkedIn Connections Scraper:

In [49]:
# Import all needed libraries
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException 
from msedge.selenium_tools import Edge, EdgeOptions

def get_connection_data(card):
    """Extract data from from connection card"""
    membersName = card.find_element_by_xpath('.//span[2]').text
    membersOccupation = card.find_element_by_xpath('.//span[4]').text
    
    data = (membersName, membersOccupation)
    return data

In [50]:
# application variables
userId = input('username: ')
my_password = getpass('Password: ')

# create instance of web driver
options= EdgeOptions()
options.use_chromium =True
driver = Edge(options=options)

# navigate to the login screen
driver.get('https://www.linkedin.com/login')
driver.maximize_window()

username = driver.find_element_by_xpath('//input[@name="session_key"]')
username.send_keys(userId)

password = driver.find_element_by_xpath('//input[@name="session_password"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)
sleep(3)

# navigate to my network 
driver.find_element_by_link_text("My Network").click()
sleep(3)
# navigate to connections
driver.find_element_by_class_name("mn-community-summary__entity-info").click()
sleep(3)

username: kh-mon-d@live.com
Password: ········


In [51]:
# get all connections on the page
data = []
last_position = driver.execute_script("return window.pageYOFFset;")
scrolling = True

while scrolling: 
    page_cards = driver.find_elements_by_class_name("mn-connection-card__details")
    for card in page_cards[-15:]:
        details = get_connection_data(card)
        data.append(details)
        
    scroll_attempt = 0
    while True:
        # check scroll postion
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOFFset;")
        
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break
            
# close the web driver
driver.close()

# Saving the Connection Data

In [52]:
with open('linkedin_connections_data.csv', mode='w', encoding='utf-8-sig', newline='\n') as csv_file:
    tableHead = ['ConnectionName', 'ConnectionOccupation']
    writer = csv.writer(csv_file)
    writer.writerow(tableHead)
    writer.writerows(data)